In [23]:
import boto3
import s3fs
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
fy15 = dd.read_parquet('s3://dask/2016_parking_violations/')

fy15

FileNotFoundError: The specified key does not exist.